# Daten aus der API von Chess.com

## 1. Runterladen und Speichern als Dataframe

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
import urllib

In [318]:
text_input = "hikaru"
username = text_input.lower() # wird aus Streamlit gezogen

In [319]:
def datenziehung(username):
        # Use of the API to download monthly archives
        baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"

        # read the archives url and store in a list
        f = urllib.request.urlopen(baseUrl + "archives")
        archives = f.read().decode("utf-8")
        archives = archives.replace("{\"archives\":[\"", "\",\"")
        archivesList = archives.split("\",\"" + baseUrl)
        archivesList[len(archivesList)-1] = archivesList[len(archivesList)-1].rstrip("\"]}")
        df_list = []
        for i in range(len(archivesList)-1):
                url = baseUrl + archivesList[i+1]
                # Make an HTTP request to the URL
                response = requests.get(url)

                # Parse the response into a Python dictionary
                data = json.loads(response.text)

                # Convert the dictionary into a list of records
                records = data['games']

                # Convert the list of records into a dataframe and append it to the list
                df_list.append(pd.DataFrame.from_records(records))

        # Concatenate all the dataframes into a single dataframe
        df = pd.concat(df_list) 
        return df

In [320]:
df = datenziehung(username)

## 2. Datenbearbeitung

In [ ]:
def datenbearbeitung(df):
    # Spalten-Titel umbennen
    dict = {'url': 'Link',
            'pgn': 'PGN',
            'time_control': 'TimeControl',
            'end_time': 'EndTime',
            'rated':'Rated',
            'start_time':'StartTime'
            }

    df.rename(columns=dict,
            inplace=True)

    df['meine Farbe'] = np.where(
        df['white'].apply(lambda x: x.get('username')) == username, 
        'Weiß', 
        'Schwarz'
        )

    df['mein Elo'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['white'].apply(lambda x: x.get('rating')), 
        df['black'].apply(lambda x: x.get('rating'))
        )

    df['Gegner Elo'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('rating')), 
        df['white'].apply(lambda x: x.get('rating'))
        )

    df['Ergebnis'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['white'].apply(lambda x: x.get('result')),
        df['black'].apply(lambda x: x.get('result'))
    )

    df['Gegner Ergebnis'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('result')),
        df['white'].apply(lambda x: x.get('result'))
    )

    df['Weiß Ergebnis'] = df['white'].apply(lambda x: x.get('result')) #drinlassen für Kontrolle

    df['Schwarz Ergebnis'] = df['black'].apply(lambda x: x.get('result')) #drinlassen für Kontrolle

    df['Gegner-Name'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('username')),
        df['white'].apply(lambda x: x.get('username'))
    )

    df['Gegner API'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['black'].apply(lambda x: x.get('@id')),
        df['white'].apply(lambda x: x.get('@id'))
    )

    game_type_mapping = {
        "rapid":"Schnellschach",
        "bullet":"Bullet",
        "blitz":"Blitz",
        "daily":"Täglich",
        
    }
    df['Spiel-Art'] = df['time_class'].astype(str).map(game_type_mapping).astype('category')

    time_control_mapping = {
        "600":"10 Minuten",
        "60":"1 Minute",
        "60+1":"1 | 1",
        "300":"5 Minuten",
        "1/172800":"2 Tage",
        "180":"3 Minuten",
        "1800":"30 Minuten",
        "120+1":"2 | 1",
        "900+5":"10 | 5",
        "900":"15 Minuten",
        "300+5":"5 | 5",
        "180+2":"3 | 2",
        "30":"30 Sek.",
        "900+10":"10 | 10",
        "1/86400":"1 Tag"
    }	
    df['Zeit']= df['TimeControl'].astype(str).map(time_control_mapping)

    df['End-Datum'] = df['EndTime'].apply(datetime.datetime.fromtimestamp)

    df.index = np.arange(1, len(df) + 1)
    
    df['Meine Genauigkeit'] = np.where(
        df.accuracies.isna(),
        {},
        df.accuracies
    )

    df['Meine Genauigkeit'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['Meine Genauigkeit'].apply(lambda x: x.get('white')),
        df['Meine Genauigkeit'].apply(lambda x: x.get('black'))   
    )

    df['Meine Genauigkeit'] =["{:.2%}".format(x/100) for x in df['Meine Genauigkeit']]

    df['Gegner Genauigkeit'] = np.where(
        df.accuracies.isna(),
        {},
        df.accuracies
    )

    df['Gegner Genauigkeit'] = np.where(
        df['meine Farbe'] == 'Weiß',
        df['Gegner Genauigkeit'].apply(lambda x: x.get('black')),
        df['Gegner Genauigkeit'].apply(lambda x: x.get('white'))   
    )
    
    df['Gegner Genauigkeit'] =["{:.2%}".format(x/100) for x in df['Gegner Genauigkeit']]

    ergebnis_mapping = {
        "checkmated":"Schachmatt",
        "timeout":"Zeitüberschreitung",
        "resigned":"Aufgabe",
        "win":"gewonnen",
        "stalemate":"Patt",
        "abandoned":"Verlassen",
        "repetition":"Stellungswiederholung",
        "timevsinsufficient":"Zeitüberschreitung gegen unzureichendes Material",
        "threecheck":"3 Mal Schach-gestellt",
        "agreed":"Vereinbarung",
        "insufficient":"Nicht genügend Material",
        "50move":"50 Züge Regel",
        "lose":"verloren",
        "kingofthehill":"Gegnerischer König hat die Mitte erreicht",
        "bughousepartnerlose":"Bughouse-Partner hat verloren"
    }	

    df['Ausgang'] = np.where(
        df['Ergebnis'] == 'win',
        'gewonnen',
        np.where(
            df['Gegner Ergebnis'] == 'win',
            'veloren',
            'unentschieden'
        )
    )

    df['Ausgang-Grund'] = np.where(
        df['Ergebnis'] == 'win',
        df['Gegner Ergebnis'].astype(str).map(ergebnis_mapping),
        df['Ergebnis'].astype(str).map(ergebnis_mapping)
    )


    ergebnis_mapping = {
        "chess":"klassisch",
        "kingofthehill":"King of the Hill",
        "threecheck":"3 Schach Schach",
        "chess960":"Schach960",
        "crazyhouse":"Einsetzschach",
        "bughouse":"Tandemschach",
        "oddschess":"Odds Chess"
    }	
    df['Variante'] = df.rules.map(ergebnis_mapping)

    return df

In [ ]:
df = datenbearbeitung(df)

In [ ]:
df

,Link,PGN,TimeControl,EndTime,Rated,accuracies,tcn,uuid,initial_setup,fen,...,Gegner-Name,Gegner API,Spiel-Art,Zeit,End-Datum,Meine Genauigkeit,Gegner Genauigkeit,Ausgang,Ausgang-Grund,Variante
1,https://www.chess.com/game/live/692667823,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1389052479,True,"{'white': 95.67516884318614, 'black': 84.33734...",mC0Kbs5Qow2Ufo92gm!0eg8!ltZRpx3Vcu6Sdl7Zgp!3sJ...,82282996-91e2-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,6k1/1p2R3/p1p5/8/2P1B3/1P1P1p1P/P6K/8 b - -,...,Hikaru,https://api.chess.com/pub/player/hikaru,Blitz,3 Minuten,2014-01-07 00:54:39,84.34%,95.68%,veloren,Aufgabe,klassisch
2,https://www.chess.com/game/live/692670646,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1389052832,True,"{'white': 93.75848310550438, 'black': 92.83682...",lB!TkAXPbs6Xnv0SmCZJCKTZAJSJftYIBI9Idm5QvDQBmE...,eba837bc-91e2-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/8/7p/7P/1p6/pBkpp2b/8/4K3 w - -,...,Godswill,https://api.chess.com/pub/player/godswill,Blitz,3 Minuten,2014-01-07 01:00:32,92.84%,93.76%,gewonnen,Zeitüberschreitung,klassisch
3,https://www.chess.com/game/live/692695595,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",60,1389054905,True,"{'white': 62.673112175007915, 'black': 93.9141...",mC0KbsZRgv!Tlt90fm8!egYQveXHowHzsbRJeoJCtCTCde...,8ce4bd6e-91e6-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnb3k1/5ppp/2p5/p1b5/1p3B2/6P1/PPPNr1KP/R4R2 w...,...,manueliux_06,https://api.chess.com/pub/player/manueliux_06,Bullet,1 Minute,2014-01-07 01:35:05,93.91%,62.67%,gewonnen,Aufgabe,klassisch
4,https://www.chess.com/game/live/692696567,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",60,1389055065,True,"{'white': 93.66040288493149, 'black': 87.02292...",mCYQbsZJgv0KvKJBsm70Kv0CltCJmB!Tfm90Br8!eg6LkA...,b14099c6-91e6-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/p7/R6p/5pk1/8/6P1/P4P2/6K1 b - -,...,Hikaru,https://api.chess.com/pub/player/hikaru,Bullet,1 Minute,2014-01-07 01:37:45,87.02%,93.66%,veloren,Aufgabe,klassisch
5,https://www.chess.com/game/live/692700868,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1389055570,True,"{'white': 91.58871391139664, 'black': 87.95104...",mCZRow2Ufo92gm!TlB8!kAYQeg5Zms0KBJZIjzIOiqOYpx...,5196be28-91e7-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2R5/1p3rkp/1B1p1bp1/p2Pp3/P3B3/6PP/5P1K/8 b - -,...,Hikaru,https://api.chess.com/pub/player/hikaru,Blitz,3 Minuten,2014-01-07 01:46:10,87.95%,91.59%,veloren,Aufgabe,klassisch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45938,https://www.chess.com/game/live/66595413697,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1672856632,True,"{'white': 95.44, 'black': 86.41}",mC0KfA!Tlt5Qbs9zgvZJCJTJegzsjs8!fe98cl3Vpx7Riy...,87fb810c-8c5c-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r5k1/1pp2pp1/pnn3rp/3q1N2/P3Q3/2PP3P/2PB1PP1/1...,...,Hikaru,https://api.chess.com/pub/player/hikaru,Blitz,3 Minuten,2023-01-04 19:23:52,86.41%,95.44%,veloren,Aufgabe,klassisch
45939,https://www.chess.com/game/live/66595818065,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1672856983,True,"{'white': 85.21, 'black': 84.9}",gv!Tjr2Ucj92lB8!muZRfm5Zeg98kAYQrzWGiq7YAIRJzH...,f6e0a288-8c5c-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,5R2/8/8/5p1p/5kr1/7K/8/8 b - -,...,Hikaru,https://api.chess.com/pub/player/hikaru,Blitz,3 Minuten,2023-01-04 19:29:43,84.90%,85.21%,veloren,Zeitüberschreitung,klassisch
45940,https://www.chess.com/game/live/66595978051,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1672857128,True,"{'white': 77.5, 'black': 92.7}",mCYQgvZJbs6EpxEvdv0SlBJCvC!TCt5Zow2Ufo92eg8!sm...,c35e7d8b-8c5d-11ed-b5e4-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2r3k1/1B3p2/1p2p1p1/2b4p/7P/1QP5/PP6/5q1K w - -,...,XDPS,https:

In [279]:
df.columns

Index(['Link', 'PGN', 'TimeControl', 'EndTime', 'Rated', 'tcn', 'uuid',
       'initial_setup', 'fen', 'time_class', 'rules', 'white', 'black',
       'StartTime', 'accuracies', 'meine Farbe', 'mein Elo', 'Gegner Elo',
       'Ergebnis', 'Gegner Ergebnis', 'Weiß Ergebnis', 'Schwarz Ergebnis',
       'Gegner-Name', 'Gegner API', 'Spiel-Art', 'Zeit', 'End-Datum',
       'Meine Genauigkeit', 'Gegner Genauigkeit', 'Ausgang', 'Ausgang-Grund'],
      dtype='object')

In [229]:
# Anzahl der Berechnungen der Genauigkeit
len(df.accuracies.dropna())

57

In [230]:
df.iloc[2813]

Link                        https://www.chess.com/game/live/65661255197
PGN                   [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
TimeControl                                                         600
EndTime                                                      1671923104
Rated                                                              True
tcn                   lB0KksKBgvBsmusjcj!TfA9zblzldl8!egYQpxZJAr6SvM...
uuid                               e56b29f8-83dd-11ed-bb21-78ac4409ff3c
initial_setup         rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...
fen                                         2k5/2Q5/2K5/8/8/8/8/8 b - -
time_class                                                        rapid
rules                                                             chess
white                 {'rating': 1018, 'result': 'win', '@id': 'http...
black                 {'rating': 991, 'result': 'checkmated', '@id':...
StartTime                                                       

In [ ]:
df.GenauigkeitWeiß = [row['white'] for row in df.accuracies.dropna()]

In [ ]:
df.GenauigkeitWeiß

In [ ]:
df.accuracies = df.accuracies.fillna('')

In [218]:
df['Meine Genauigkeit'].unique()


array(['nan%', '91.25%', '46.14%', '56.61%', '71.42%', '53.45%', '68.40%',
       '53.05%', '62.62%', '59.56%', '59.40%', '71.43%', '83.56%',
       '69.03%', '42.24%', '69.66%', '68.26%', '66.71%', '83.58%',
       '78.19%', '80.62%', '84.77%', '71.67%', '66.27%', '53.20%',
       '72.25%', '71.99%', '57.36%', '76.06%', '69.56%', '75.23%',
       '55.09%', '75.35%', '58.33%', '70.39%', '76.34%', '73.51%',
       '71.91%', '39.58%', '54.94%', '61.27%', '40.51%', '70.66%',
       '49.17%', '88.51%', '71.20%', '73.54%', '55.93%', '95.40%',
       '76.47%', '75.75%', '62.24%', '83.49%', '75.36%', '82.67%',
       '58.91%', '83.18%', '74.90%'], dtype=object)

In [ ]:
dicc = {'white': 68.4, 'black': 57.87}

In [ ]:
dicc.get('white')

In [ ]:
[row['white'] for row in df['accuracies']]


In [ ]:
df['Weiße Genauigkeit'] = df.accuracies.get('white')

In [ ]:
df['Weiße Genauigkeit'].unique()

In [ ]:
df['Meine Genauigkeit'].unique()

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['black', 'white', 'EndTime', 'uuid'])

In [272]:
df.groupby(['Ergebnis','Gegner Ergebnis', 'Ausgang-Grund','Ausgang']).size().reset_index().rename(columns={0:'count'})

,Ergebnis,Gegner Ergebnis,Ausgang-Grund,Ausgang,count
0,50move,50move,50 Züge Regel,unentschieden,1
1,abandoned,win,Verlassen,veloren,32
2,agreed,agreed,Vereinbarung,unentschieden,35
3,checkmated,win,Schachmatt,veloren,621
4,insufficient,insufficient,Nicht genügend Material,unentschieden,13
5,repetition,repetition,Stellungswiederholung,unentschieden,28
6,resigned,win,Aufgabe,veloren,638
7,stalemate,stalemate,Patt,unentschieden,41
8,threecheck,win,3 Mal Schach-gestellt,veloren,3
9,timeout,win,Zeitüberschreitung,veloren,110


In [ ]:
Event
Spiel-Art
--Variante
Zeit
Start-Datum
End-Datum
Dauer in Minuten




Anzahl Züge
ECO
Uhrzeit
Wochentag
Spielzüge

In [297]:
df.columns

Index(['Link', 'PGN', 'TimeControl', 'EndTime', 'Rated', 'accuracies', 'tcn',
       'uuid', 'initial_setup', 'fen', 'time_class', 'rules', 'white', 'black',
       'meine Farbe', 'mein Elo', 'Gegner Elo', 'Ergebnis', 'Gegner Ergebnis',
       'Weiß Ergebnis', 'Schwarz Ergebnis', 'Gegner-Name', 'Gegner API',
       'Spiel-Art', 'Zeit', 'End-Datum', 'Meine Genauigkeit',
       'Gegner Genauigkeit', 'Ausgang', 'Ausgang-Grund', 'Variante'],
      dtype='object')

In [ ]:
df.Zeit.unique()

array(['180', '60', '1/259200', '1/1209600', '1/172800', '1/86400',
       '1/432000', '1/604800', '180+2', '300+1', '180+1', '60+1', '600',
       '300+2', '1800', '900', '300', '900+10', '180+3', '2700+15',
       '900+2', '10', '120+1', '600+2', '300+5', '600+5', '30', '300+10',
       '2700', '120', '1500+10', '600+3', '900+3', '240', '720',
       '3600+45', '300+3', '600+10', '480+2', '180+5', '480', '1200+10',
       '420', '1200'], dtype=object)

In [ ]:
time_control_mapping = {
    "10":"10 Sek.",
    "30":"30 Sek.",
    "60":"1 Minute",
    "60+1":"1 | 1",
    "120+1":"2 | 1",

    "180":"3 Minuten",
    "180+1":"3 | 1",
    "180+2":"3 | 2",
    "180+3":"3 | 3",
    "180+5":"3 | 5",
    "300":"5 Minuten",
    "300+1":"5 | 1",
    "300+2":"5 | 2",
    "300+3":"5 | 3",
    "300+5":"5 | 5",
    "300+10":"5 | 10",
    "420":"7 Minuten",
    "480":"8 Minuten",
    "480+2":"8 | 2",

    "600":"10 Minuten",
    "600+2":"10 | 2",
    "600+3":"10 | 3",
    "600+5":"10 | 5",
    "600+10":"10 | 10",
    "900":"15 Minuten",
    "900+2":"15 | 2",
    "900+3":"15 | 3",
    "900+5":"15 | 5",
    "900+10":"15 | 10",
    "1200":"20 Minuten",
    "1200+10":"20 | 10",
    "1800":"30 Minuten",
    "2700+15":"45 | 15",
    "3600+45":"60 | 45",

    "1/86400":"1 Tag",
    "1/172800":"2 Tage",
    "1/259200":"3 Tage",
    "1/432000":"5 Tage",
    "1/604800":"7 Tage",
    "1/1209600":"14 Tage"
}	
df['Zeit']= df['TimeControl'].astype(str).map(time_control_mapping)

In [317]:
df.head()

,Link,PGN,TimeControl,EndTime,Rated,accuracies,tcn,uuid,initial_setup,fen,...,Gegner-Name,Gegner API,Spiel-Art,Zeit,End-Datum,Meine Genauigkeit,Gegner Genauigkeit,Ausgang,Ausgang-Grund,Variante
1,https://www.chess.com/game/live/692667823,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1389052479,True,"{'white': 95.67516884318614, 'black': 84.33734...",mC0Kbs5Qow2Ufo92gm!0eg8!ltZRpx3Vcu6Sdl7Zgp!3sJ...,82282996-91e2-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,6k1/1p2R3/p1p5/8/2P1B3/1P1P1p1P/P6K/8 b - -,...,Hikaru,https://api.chess.com/pub/player/hikaru,Blitz,3 Minuten,2014-01-07 00:54:39,84.34%,95.68%,veloren,Aufgabe,klassisch
2,https://www.chess.com/game/live/692670646,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1389052832,True,"{'white': 93.75848310550438, 'black': 92.83682...",lB!TkAXPbs6Xnv0SmCZJCKTZAJSJftYIBI9Idm5QvDQBmE...,eba837bc-91e2-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/8/7p/7P/1p6/pBkpp2b/8/4K3 w - -,...,Godswill,https://api.chess.com/pub/player/godswill,Blitz,3 Minuten,2014-01-07 01:00:32,92.84%,93.76%,gewonnen,Zeitüberschreitung,klassisch
3,https://www.chess.com/game/live/692695595,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",60,1389054905,True,"{'white': 62.673112175007915, 'black': 93.9141...",mC0KbsZRgv!Tlt90fm8!egYQveXHowHzsbRJeoJCtCTCde...,8ce4bd6e-91e6-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnb3k1/5ppp/2p5/p1b5/1p3B2/6P1/PPPNr1KP/R4R2 w...,...,manueliux_06,https://api.chess.com/pub/player/manueliux_06,Bullet,1 Minute,2014-01-07 01:35:05,93.91%,62.67%,gewonnen,Aufgabe,klassisch
4,https://www.chess.com/game/live/692696567,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",60,1389055065,True,"{'white': 93.66040288493149, 'black': 87.02292...",mCYQbsZJgv0KvKJBsm70Kv0CltCJmB!Tfm90Br8!eg6LkA...,b14099c6-91e6-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/p7/R6p/5pk1/8/6P1/P4P2/6K1 b - -,...,Hikaru,https://api.chess.com/pub/player/hikaru,Bullet,1 Minute,2014-01-07 01:37:45,87.02%,93.66%,veloren,Aufgabe,klassisch
5,https://www.chess.com/game/live/692700868,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,1389055570,True,"{'white': 91.58871391139664, 'black': 87.95104...",mCZRow2Ufo92gm!TlB8!kAYQeg5Zms0KBJZIjzIOiqOYpx...,5196be28-91e7-11de-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2R5/1p3rkp/1B1p1bp1/p2Pp3/P3B3/6PP/5P1K/8 b - -,...,Hikaru,https://api.chess.com/pub/player/hikaru,Blitz,3 Minuten,2014-01-07 01:46:10,87.95%,91.59%,veloren,Aufgabe,klassisch
